<a href="https://colab.research.google.com/github/macuriels/umass_dacss_datastorytelling/blob/main/international_sanctions_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
import spacy

In [3]:
url = 'https://raw.githubusercontent.com/macuriels/umass_dacss_datastorytelling/main/GSDB_V3.csv'

In [4]:
df = pd.read_csv(url)

df

,case_id,sanctioned_state,sanctioning_state,begin,end,trade,descr_trade,arms,military,financial,travel,other,target_mult,sender_mult,objective,success
0,1,German Democratic Republic,Germany,1949,1973,0,NaN,0,0,0,0,1,0,0,territorial_conflict,success_total
1,2,Pakistan,India,1949,1951,1,"exp_compl, imp_compl",0,0,0,0,0,0,0,policy_change,nego_settlement
2,3,Bulgaria,United States,1950,1966,0,NaN,0,0,0,0,1,0,0,destab_regime,failed
3,4,Bulgaria,United States,1950,1959,0,NaN,0,0,0,1,0,0,0,destab_regime,success_part
4,5,Bulgaria,United States,1950,1963,0,NaN,0,0,1,0,0,0,0,destab_regime,success_part
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1320,1321,Uganda,United States,2022,2022,1,"exp_part, imp_part",0,0,1,0,0,0,0,policy_change,ongoing
1321,1322,Ukraine,United Kingdom,2022,2022,0,NaN,0,0,1,0,0,0,0,end_war,ongoing
1322,1323,United Arab Emirates,United States,2022,2022,1,"exp_part, imp_part",0,0,1,0,0,0,0,terrorism,ongoing
1323,1324,United States,China,2022,2022,0,NaN,0,0,0,1,0,0,0,policy_change,ongoing


In [5]:
# Replace specific patterns in 'sanctioned_state' and 'sanctioning_state' columns
replace_dict = {
    'Congo, Democratic Republic of the, Eritrea, Ethiopia (excludes Eritrea), Kenya, Rwanda, Tanzania, Uganda, Zambia': 'Democratic Republic of the Congo, Eritrea, Ethiopia (excludes Eritrea), Kenya, Rwanda, Tanzania, Uganda, Zambia'
    ,'Congo, Democratic Republic of the': 'Democratic Republic of the Congo'
    ,'Korea, North': 'North Korea'
    ,'Korea, South': 'South Korea'
    ,'Yemen, North': 'North Yemen'
    ,'Gambia, The': 'The Gambia'
    ,'Egypt, Arab Rep.': 'Egypt'
    ,'Egypt, Arab Rep., Syria': 'Egypt, Syria'
    ,'Saudi Arabia, United Arab Emirates, Bahrain, Egypt, Arab Rep.': 'Saudi Arabia, United Arab Emirates, Bahrain, Egypt'
}

df['sanctioned_state'].replace(replace_dict, inplace=True)
df['sanctioning_state'].replace(replace_dict, inplace=True)

In [6]:
# Function to split values in 'sanctioned_state' into separate rows
def split_countries_and_institutions(row):
    countries_institutions = row['sanctioned_state'].split(',')
    if len(countries_institutions) > 1:
        rows = []
        for country_institution in countries_institutions:
            new_row = row.copy()
            new_row['sanctioned_state'] = country_institution
            rows.append(new_row)
        return rows
    return [row]  # Wrap the single row in a list

# Apply the function and create a list of dictionaries
rows_list = []
for index, row in df.iterrows():
    rows_list.extend(split_countries_and_institutions(row))

# Create a new DataFrame
df = pd.DataFrame(rows_list)

# Reset the index of the new DataFrame
df = df.reset_index(drop=True)

In [8]:
# Function to split values in 'sanctioning_state' into separate rows
def split_countries_and_institutions_column2(row):
    countries_institutions = row['sanctioning_state'].split(',')
    if len(countries_institutions) > 1:
        rows = []
        for country_institution in countries_institutions:
            new_row = row.copy()
            new_row['sanctioning_state'] = country_institution
            rows.append(new_row)
        return rows
    return [row]  # Wrap the single row in a list

# Apply the function and create a list of dictionaries
rows_list_column2 = []
for index, row in df.iterrows():
    rows_list_column2.extend(split_countries_and_institutions_column2(row))

# Create a new DataFrame for the second column
df = pd.DataFrame(rows_list_column2)

# Reset the index of the new DataFrame
df = df.reset_index(drop=True)

In [9]:
# Load the spaCy model
nlp = spacy.load("en_core_web_sm")

def filter_organizations(row):
    for col in ['sanctioned_state', 'sanctioning_state']:
        # Check if entry is all upper case
        if row[col].isupper():
            return False

        # Check if entry contains a number
        if any(char.isdigit() for char in row[col]):
            return False

        # Check for specific entries to exclude
        if row[col] == "League of Arab States":
            return False

        # Check if entry contains the word "Balkan"
        if "Balkan" in row[col]:
            return False

        # Continue checking for organization entities using spaCy
        doc = nlp(row[col])
        for ent in doc.ents:
            if ent.label_ == 'ORG':
                return False

    return True

# Apply the function to filter out organization rows
df = df[df.apply(filter_organizations, axis=1)]

In [10]:
# Remove leading white spaces from all cells
df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)

In [11]:
df

,case_id,sanctioned_state,sanctioning_state,begin,end,trade,descr_trade,arms,military,financial,travel,other,target_mult,sender_mult,objective,success
0,1,German Democratic Republic,Germany,1949,1973,0,NaN,0,0,0,0,1,0,0,territorial_conflict,success_total
1,2,Pakistan,India,1949,1951,1,"exp_compl, imp_compl",0,0,0,0,0,0,0,policy_change,nego_settlement
2,3,Bulgaria,United States,1950,1966,0,NaN,0,0,0,0,1,0,0,destab_regime,failed
3,4,Bulgaria,United States,1950,1959,0,NaN,0,0,0,1,0,0,0,destab_regime,success_part
4,5,Bulgaria,United States,1950,1963,0,NaN,0,0,1,0,0,0,0,destab_regime,success_part
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1885,1321,Uganda,United States,2022,2022,1,"exp_part, imp_part",0,0,1,0,0,0,0,policy_change,ongoing
1886,1322,Ukraine,United Kingdom,2022,2022,0,NaN,0,0,1,0,0,0,0,end_war,ongoing
1887,1323,United Arab Emirates,United States,2022,2022,1,"exp_part, imp_part",0,0,1,0,0,0,0,terrorism,ongoing
1888,1324,United States,China,2022,2022,0,NaN,0,0,0,1,0,0,0,policy_change,ongoing
